# Create point distributions on a sphere

A couple of example to create point distributions on a sphere

### Example #1 Points on a sphere using _healpy_
HEALPix is an acronym for Hierarchical Equal Area isoLatitude Pixelation of a sphere. The code below makes use of the healpy python module, available here:  
https://healpy.readthedocs.io/en/latest/  
to generate a set of evenly distributed points across the sphere.

The density of the points is set by the nSide parameter.

The resulting feature is written to a gpmlz file - loading this file into GPlates is the easiest way to see what the distribution looks like.

In [8]:
import pygplates
import healpy as hp
import numpy as np


### Make a multipoint feature
### with points evenly distributed points on the sphere
nSide = 32
othetas,ophis = hp.pix2ang(nSide,np.arange(12*nSide**2))
othetas = np.pi/2-othetas
ophis[ophis>np.pi] -= np.pi*2

lats = np.degrees(othetas) 
lons = np.degrees(ophis)

# The next line exlicitly creates the feature as a 'MeshNode' gpml type, so that
# GPlates will display velocities at each point
multipoint_feature = pygplates.Feature(
    pygplates.FeatureType.create_from_qualified_string('gpml:MeshNode'))
multipoint = pygplates.MultiPointOnSphere(zip(lats,lons))  
multipoint_feature.set_geometry(multipoint)
multipoint_feature.set_name('Equal Area points from healpy')

output_feature_collection = pygplates.FeatureCollection(multipoint_feature)
    
output_feature_collection.write('healpix_mesh_feature.gpmlz')



### Example #2

Generate a multipoint feature with a pseudo-random distribution on the sphere. Again, the result is saved to gpml so you can look at it in GPlates. If the points are too dense, or not dense enough, simply change N and run again.

In [11]:
# N is simply the total number of points that we want to create
N = 20000

## Marsaglia's method
dim = 3

norm = np.random.normal
normal_deviates = norm(size=(dim, N))

radius = np.sqrt((normal_deviates**2).sum(axis=0))
points = normal_deviates/radius

# The above code returns points on a sphere, but specified in 3D cartesian
# space rather than lat/long space. However, we can use these directly to 
# create the multipoint feature, pygplates will recognise them as x,y,z

multipoint_feature = pygplates.Feature()
multipoint_feature.set_geometry(pygplates.MultiPointOnSphere((points.T)))
multipoint_feature.set_name('Random Points from Marsaglia''s method')

multipoint_feature_collection = pygplates.FeatureCollection(multipoint_feature)

multipoint_feature_collection.write('pseudo_random_points_feature.gpmlz')
